In [2]:
# !pip install llama-index
# !pip install llama-index-llms-ollama
# !pip install llama-index-llms-replicate
# !pip install llama-index-embeddings-huggingface
# !pip install llama-parse
# # !pip install replicate

In [3]:
# from llama_index.llms.ollama import Ollama

# llm = Ollama(model="llama3", request_timeout=120.0)

/opt/anaconda3/envs/finance_tracker_app/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

/opt/anaconda3/envs/finance_tracker_app/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [8]:
# from llama_parse import LlamaParse

# docs = LlamaParse(result_type="text").load_data("../data/2024_04.pdf")
# docs_kendrick = LlamaParse(result_type="text").load_data("./data/kendrick.pdf")
# docs_drake = LlamaParse(result_type="text").load_data("./data/drake.pdf")
# docs_both = LlamaParse(result_type="text").load_data(
#     "./data/drake_kendrick_beef.pdf"
# )


from llama_index.core import SimpleDirectoryReader
docs = SimpleDirectoryReader(input_files=["../data/bank_statements/east_west_bank/2024_04.pdf"]).load_data()

# docs_kendrick = SimpleDirectoryReader(input_files=["data/kendrick.pdf"]).load_data()
# docs_drake = SimpleDirectoryReader(input_files=["data/drake.pdf"]).load_data()
# docs_both = SimpleDirectoryReader(input_files=["data/drake_kendrick_beef.pdf"]).load_data()

In [9]:
docs

[Document(id_='92845bce-e839-4f75-8e7b-bd1b92ac2da1', embedding=None, metadata={'page_label': '1', 'file_name': '2024_04.pdf', 'file_path': '../data/bank_statements/east_west_bank/2024_04.pdf', 'file_type': 'application/pdf', 'file_size': 1749727, 'creation_date': '2024-05-25', 'last_modified_date': '2024-05-25'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='ACCOUNT STATEMENT\n   ( 0)\nJARED VINCENT YU 44357 VIEW POINT CIR\n FREMONT CA94539-6258\n \nCREDITS\nDEBITS\nDAILY BALANCES\nOVERDRAFT/RETURN ITEM FEESNumber Date Transaction Description Additions\nDate Transaction Description Subtractions\nDate Amount Date Amount Date AmountAccount number     26-27011667\nLow balance        $2,982.74\nAverage balance        $3,230.76Beginning balance          $3

In [10]:
response = llm.complete("Do you see the pdf document I loaded yet?")

print(response)

ReadTimeout: timed out

In [1]:
import pdftotext
import os
import re
import pandas as pd

In [2]:
# EDGE CASE: MULTI-PAGE STATEMENTS
# EDGE CASE: MULTI-LINE TRANSACTIONS

In [59]:
def pair_sequentially(numbers):
    # Check if the list has an even length
    if len(numbers) % 2 != 0:
        raise ValueError("The list must have an even length")

    # Create the list of pairs
    paired_list = []
    for i in range(0, len(numbers), 2):
        paired_list.append(numbers[i:i+2])
    
    return paired_list

def remove_indices(lst, indices):
    # Sort indices in descending order to avoid reindexing issues
    indices = sorted(indices, reverse=True)
    
    # Remove elements at each index
    for index in indices:
        if 0 <= index < len(lst):
            lst.pop(index)
        else:
            raise IndexError(f"Index {index} is out of bounds for list of length {len(lst)}")
    
    return lst

def has_consecutive_values(data):
  """
  This function checks if a list contains any consecutive values (adjacent duplicates).

  Args:
      data: A list of any data type.

  Returns:
      True if there are consecutive values, False otherwise.
  """
  if len(data) <= 1:
    return False  # Need at least 2 elements for consecutive values

  # Iterate through the list, checking for adjacent duplicates
  for i in range(1, len(data)):
    if data[i] == data[i-1] + 1:
      return True
  return False

def concatenate_multi_line_transactions(transaction_lines_list, bank, year, month):
    """
    Edit credit and debit line lists to concatenate strings from multi-line items.
    """
    transaction_lines_list = transaction_lines_list.copy()
    date_pattern = r"^\d{2}-\d{2}$"
    bad_idx_list = []
    for idx, txc_line in enumerate(transaction_lines_list):
        if not bool(re.match(date_pattern, txc_line[:5])):
            bad_idx_list.append(idx)

    if has_consecutive_values(data=bad_idx_list):
        raise Exception(f'Transactions with more than two lines found. Info: {bank}, {year}, {month}')

    if len(bad_idx_list) > 0:
        complete_bad_idx_list = []
        for i in bad_idx_list:
            complete_bad_idx_list.append(i - 1)
            complete_bad_idx_list.append(i)

        paired_short_line_list = pair_sequentially(complete_bad_idx_list)

        # extract them from the original list and concatenate them together
        combined_line_list = []
        for paired_lines in paired_short_line_list:
            combined_line = ''.join(transaction_lines_list[paired_lines[0]:paired_lines[1]+1])
            combined_line_list.append(combined_line)

        # drop the original items
        transaction_lines_list = remove_indices(transaction_lines_list, complete_bad_idx_list)

        return transaction_lines_list + combined_line_list
    else:
        return transaction_lines_list

def parse_lines_with_regex(lines):
    # Ref.: https://levelup.gitconnected.com/creating-a-bank-statement-parser-with-python-9223b895ebae
    transactions = []
    transaction_pattern = (
            r"(?P<transaction_date>\d+-\d+)\s*"
            r"(?P<description>.*?)\s*"
            r"(?P<amount>[\d.,]+)$"
    )

    for line in lines:
        match = re.search(pattern=transaction_pattern, string=line)
        if match:
            transactions.append(match.groupdict())

    return pd.DataFrame(transactions)

def parse_bank_statements_by_year(bank, year, bank_statements_file_path):
    """
    Go through a year of monthly bank statements for a given bank and parse
    the statements and return a df.
    """
    file_path = os.path.join(bank_statements_file_path, bank, year)
    monthly_bank_statement_list = os.listdir(file_path)
    monthly_bank_statement_list = [i for i in monthly_bank_statement_list if i != '.DS_Store']
    transactions_df_list = []

    for monthly_bank_statement_file in monthly_bank_statement_list:
        month = monthly_bank_statement_file.split('_')[1].split('.')[0]
        with open(os.path.join(file_path, monthly_bank_statement_file), "rb") as file:
            pdf = pdftotext.PDF(file, physical=True)
            if len(pdf) == 2:
                first_page = pdf[0]
                lines1 = first_page.split("\n")
                lines1 = [i.lstrip() for i in lines1]
                lines1 = [i for i in lines1 if i != '']
                lines2 = None
            elif len(pdf) == 3:
                first_page = pdf[0]
                second_page = pdf[1]
                lines1 = first_page.split("\n")
                second_page = pdf[1]
                lines2 = second_page.split("\n")

                lines1 = [i.lstrip() for i in lines1]
                lines1 = [i for i in lines1 if i != '']
                lines2 = [i.lstrip() for i in lines2]
                lines2 = [i for i in lines2 if i != '']
            else:
                pdf_length = len(pdf)
                raise Exception(f'New length for pdf ({pdf_length}), time to set new rules. Info: {bank}, {year}, {month}')
            
            # for line in lines1:
            #     print(line.lstrip())

        try:
            credits_line_idx = lines1.index('CREDITS')
            credit_balance_exists = True
        except:
            credit_balance_exists = False
        try:
            debits_line_idx = lines1.index('DEBITS')
            debits_line_on_first_page = True
        except:
            debits_line_on_first_page = False
            raise Exception(f'DEBITS not on first page, time to set new rules. Info: {bank}, {year}, {month}')

        try:
            daily_balances_line_idx = lines1.index('DAILY BALANCES')
            daily_balances_on_first_page = True
        except:
            daily_balances_line_idx = lines2.index('DAILY BALANCES')
            daily_balances_on_first_page = False
        credit_lines = lines1[credits_line_idx + 2:debits_line_idx]

        try: # check if the DEBITS balance is multi-page
            if len([i for i in lines2 if i[:4] == 'Date']) > 1:
                debit_balance_multi_page = True
            else:
                debit_balance_multi_page = False
        except:
            debit_balance_multi_page = False

        # No CREDITS and DEBITS does not extend to second
        if not credit_balance_exists and not debit_balance_multi_page:
            debit_lines = lines1[debits_line_idx + 2:daily_balances_line_idx]
            debit_lines = concatenate_multi_line_transactions(debit_lines, bank, year, month)
            debit_df = parse_lines_with_regex(lines=debit_lines)
            debit_df['amount'] = debit_df['amount'].apply(lambda x: -1 * float(x.replace(',' , '')))

            debit_df['bank'] = bank
            debit_df['year'] = year
            debit_df['month'] = month
            transactions_df_list.append(debit_df)
            continue
        # DAILY BALANCES on first page and DEBITS does not extend to second
        elif daily_balances_on_first_page and not debit_balance_multi_page:
            debit_lines = lines1[debits_line_idx + 2:daily_balances_line_idx]
        # DEBITS on first page but extends to second
        elif debits_line_on_first_page and debit_balance_multi_page:
            second_page_debit_date_line_idx = lines2.index([i for i in lines2 if i[:4] == 'Date'][0])
            debit_lines = lines1[debits_line_idx + 2:] + lines2[second_page_debit_date_line_idx + 1:daily_balances_line_idx]
        # DAILY BALANCES not on first page and DEBITS does not extend to second
        elif not daily_balances_on_first_page and not debit_balance_multi_page:
            debit_lines = lines1[debits_line_idx + 2:]
        else:
            raise Exception(f'Uncaught case. Info: {bank}, {year}, {month}')

        credit_lines = concatenate_multi_line_transactions(credit_lines, bank, year, month)
        debit_lines = concatenate_multi_line_transactions(debit_lines, bank, year, month)

        credit_df = parse_lines_with_regex(lines=credit_lines)
        debit_df = parse_lines_with_regex(lines=debit_lines)

        credit_df['amount'] = credit_df['amount'].apply(lambda x: float(x.replace(',' , '')))
        debit_df['amount'] = debit_df['amount'].apply(lambda x: -1 * float(x.replace(',' , '')))

        transactions_df = pd.concat([credit_df, debit_df])

        transactions_df['bank'] = bank
        transactions_df['year'] = year
        transactions_df['month'] = month
        transactions_df_list.append(transactions_df)

    combined_transactions_df = pd.concat(transactions_df_list)

    return combined_transactions_df

In [60]:
# transactions_2019_df = parse_bank_statements_by_year(
#     bank='east_west_bank',
#     year='2019',
#     bank_statements_file_path='../data/bank_statements',
# )

transactions_2020_df = parse_bank_statements_by_year(
    bank='east_west_bank',
    year='2020',
    bank_statements_file_path='../data/bank_statements',
)

transactions_2021_df = parse_bank_statements_by_year(
    bank='east_west_bank',
    year='2021',
    bank_statements_file_path='../data/bank_statements',
)

transactions_2022_df = parse_bank_statements_by_year(
    bank='east_west_bank',
    year='2022',
    bank_statements_file_path='../data/bank_statements',
)

transactions_2023_df = parse_bank_statements_by_year(
    bank='east_west_bank',
    year='2023',
    bank_statements_file_path='../data/bank_statements',
)

transactions_2024_df = parse_bank_statements_by_year(
    bank='east_west_bank',
    year='2024',
    bank_statements_file_path='../data/bank_statements',
)

In [93]:
total_transactions_df = pd.concat([
    transactions_2020_df,
    transactions_2021_df,
    transactions_2022_df,
    transactions_2023_df,
    transactions_2024_df
])

total_transactions_df['transaction_date_month'] = total_transactions_df['transaction_date'].apply(lambda x: x.split('-')[0])
assert sum(total_transactions_df['transaction_date_month'] != total_transactions_df['month']) == 0
total_transactions_df['transaction_date'] = total_transactions_df['year'] + '-' + total_transactions_df['transaction_date']
total_transactions_df.drop(['year', 'month', 'transaction_date_month'], inplace=True, axis=1)
total_transactions_df.sort_values(by=['transaction_date', 'amount', 'description'], ascending=[True, False, True], inplace=True)
total_transactions_df.reset_index(drop=True, inplace=True)

In [64]:
# beginning_balance = 3046.51 # 2982.74
# beginning_balance = 14552.04 # 2982.74
# beginning_balance = 15619.65 # 2982.74
beginning_balance = 2459.25 # 2982.74
round(beginning_balance + sum(total_transactions_df['amount']), 2)

2982.74

In [94]:
total_transactions_df

,transaction_date,description,amount,bank
0,2020-01-02,Deposit,510.00,east_west_bank
1,2020-01-06,Preauth Debit VENMO PAYMENT 200106,-160.00,east_west_bank
2,2020-01-06,Preauth Debit VENMO PAYMENT 200106,-180.00,east_west_bank
3,2020-01-06,Preauth Debit VENMO PAYMENT 200106,-240.00,east_west_bank
4,2020-01-06,Preauth Debit VENMO PAYMENT 200106,-370.00,east_west_bank
...,...,...,...,...
621,2024-04-08,Pre-Auth Credit CPRIME INC DIRECT ...,5839.26,east_west_bank
622,2024-04-08,Preauth Debit GOLDMAN SACHS BA COLLECTI...,-6300.00,east_west_bank
623,2024-04-23,Pre-Auth Credit CPRIME INC DIRECT ...,3235.25,east_west_bank
624,2024-04-23,Preauth Debit GOLDMAN SACHS BA COLLECTI...,-3000.00,east_west_bank


In [221]:
# build some unit tests
# create truth table, check for regressions
# rules:
# pdf generally has length +1, ignore last blank page
# CREDITS should be in first page
# DEBITS should be in first page (could be in second, haven't seen it)
# DAILY BALANCES could be in first or second
# some transactions are multi-line
# capture the transactions under CREDITS and DEBITS correctly
